<a href="https://colab.research.google.com/github/rohitkhadka1/AI-ML-DS/blob/main/llm_app_for_creative_mathematics_problem_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes torch

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import asyncio
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [ ]:
# Load Mistral-7B (quantized to 4-bit)
model_1_name = "mistralai/Mistral-7B-v0.1"
tokenizer_1 = AutoTokenizer.from_pretrained(model_1_name)

model_1 = AutoModelForCausalLM.from_pretrained(
    model_1_name,
    torch_dtype=torch.float16,  # Use FP16 for lower VRAM usage
    load_in_4bit=True,  # Quantization for memory efficiency
    device_map="auto"   # Automatically place model parts across CPU/GPU
)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Load a second model (Example: LLaMA-2-7B)
model_2_name = "meta-llama/Llama-2-7b-hf"
tokenizer_2 = AutoTokenizer.from_pretrained(model_2_name)

model_2 = AutoModelForCausalLM.from_pretrained(
    model_2_name,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
summarizer_model = model_1

NameError: name 'model_1' is not defined

In [ ]:
prompt_template = PromptTemplate(
    template="Generate distinct mathematics problems based on the following user input: {question}. Ensure that:\n"
             "- Each problem covers a different mathematical concept or difficulty level.\n"
             "- The problems include a mix of algebra, calculus, and probability where applicable.\n"
             "- The wording of each problem is unique to avoid redundancy.\n"
             "\nUser Input: {question}\nGenerated Problems:",
    input_variables=["question"]
)

summary_template = PromptTemplate(
    template="Given the following set of generated math problems, improve them by:\n"
             "- Removing redundant or repetitive phrasing.\n"
             "- Making each problem clearer and more engaging.\n"
             "- Ensuring logical progression in difficulty.\n"
             "- Formatting them properly for better readability.\n"
             "\nGenerated Problems: {responses}\nRefined Problems:",
    input_variables=["responses"]
)


In [ ]:
# Create HuggingFacePipeline instances for both models
llm_1 = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model_1,
    tokenizer=tokenizer_1,
    torch_dtype=torch.float16,
    device_map="auto",
))

Device set to use cuda:0
<ipython-input-14-6271e4592eab>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_1 = HuggingFacePipeline(pipeline=pipeline(


In [ ]:
llm_2 = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model_2,
    tokenizer=tokenizer_2,
    torch_dtype=torch.float16,
    device_map="auto",
))

Device set to use cuda:0


In [ ]:
# Define LLM Chains
chain_1 = LLMChain(llm = llm_1, prompt = prompt_template)
chain_2 = LLMChain(llm = llm_2, prompt = prompt_template)
summary_chain = LLMChain(llm = llm_1, prompt = summary_template)


<ipython-input-16-4bca17f2fc86>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_1 = LLMChain(llm = llm_1, prompt = prompt_template)


In [ ]:
# Async Execution
async def run_models(question):
    response_1_task = asyncio.create_task(chain_1.arun(question=question))
    response_2_task = asyncio.create_task(chain_2.arun(question=question))

    response_1, response_2 = await asyncio.gather(response_1_task, response_2_task)
    summary = await summary_chain.arun(responses=f"Model 1: {response_1}\nModel 2: {response_2}")

    return {"model_1": response_1, "model_2": response_2, "summary": summary}

In [ ]:
# Example Usage
user_query = "Create calculus problems involving differentiation and real-world applications."

responses = await run_models(user_query)

/usr/lib/python3.11/asyncio/events.py:84: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  self._context.run(self._callback, *self._args)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Print Outputs
print("\nModel 1 Response:\n", responses["model_1"])


Model 1 Response:
 Generate distinct mathematics problems based on the following user input: Create calculus problems involving differentiation and real-world applications.. Ensure that:
- Each problem covers a different mathematical concept or difficulty level.
- The problems include a mix of algebra, calculus, and probability where applicable.
- The wording of each problem is unique to avoid redundancy.

User Input: Create calculus problems involving differentiation and real-world applications.
Generated Problems:
1. Find the derivative of the function f(x) = x^2 + 3


In [ ]:
print("\nModel 2 Response:\n", responses["model_2"])


Model 2 Response:
 Generate distinct mathematics problems based on the following user input: Create calculus problems involving differentiation and real-world applications.. Ensure that:
- Each problem covers a different mathematical concept or difficulty level.
- The problems include a mix of algebra, calculus, and probability where applicable.
- The wording of each problem is unique to avoid redundancy.

User Input: Create calculus problems involving differentiation and real-world applications.
Generated Problems:
- Each problem covers a different mathematical concept or difficulty level.
- The problems include a mix of algebra, calculus, and probability where applicable.
- The wording of each problem is unique to avoid redundancy.

### Examples of generated problems
- "Find the amount of water lost in the first 30 minutes of a 20-minute shower if the shower uses 10 gallons per minute."
- "How many years will it take to save $2,000 if you put $100 in a savings account that pays 2% i

In [ ]:
print("\nFinal Summary:\n", responses["summary"])

NameError: name 'responses' is not defined